# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import zipfile

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
# unzip files
zip_ref = zipfile.ZipFile('Orders.zip', 'r')
zip_ref.extractall()
zip_ref.close()

orders = pd.read_csv('Orders.csv')
orders.head()


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here
# Aggregate customers by amount spent
df = orders.groupby(['CustomerID']).sum().reset_index()   # Never want to have index problems again!!!
df.head()

,CustomerID,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
0,12346,61619,541431,2011,1,2,10,74215,1.04,77183.60
1,12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00
2,12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24
3,12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55
4,12350,1365627,9231629,34187,34,51,272,197,65.30,334.40


In [4]:
# Calculate the Quantiles of each customer
df['Quantiles'] = pd.qcut(df['amount_spent'], q=[0,0.75,0.95,1] ,precision=0, labels = ['Low_tear','Preferred', 'VIP'])
df[df['Quantiles'].notna()].sort_values(['amount_spent'], ascending=False)

,CustomerID,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent,Quantiles
1690,14646,544561120,1163267611,4182810,14191,6552,24488,197491,5176.09,280206.02,VIP
4202,18102,138022684,243297801,866723,3746,1261,5587,64124,1940.92,259657.30,VIP
3729,17450,92683919,188845149,677704,2292,842,4140,69993,1143.32,194550.79,VIP
3009,16446,929130,1688629,6033,22,11,27,80997,4.98,168472.50,VIP
1880,14911,1737367680,3196374868,11416155,46220,18930,68148,80515,26185.72,143825.06,VIP
...,...,...,...,...,...,...,...,...,...,...,...
4099,17956,144967,548862,2011,4,1,13,1,12.75,12.75,Low_tear
3015,16454,802063,1142747,4022,20,7,27,3,3.95,6.90,Low_tear
1794,14792,766395,1140006,4022,20,10,18,2,6.20,6.20,Low_tear
3218,16738,90155,543986,2011,2,2,9,3,1.25,3.75,Low_tear


In [53]:
# Check if it all makes sense
#Table with Types of customers
type_customer_df = df.groupby(['Quantiles']).sum()
type_customer_df = type_customer_df.drop(['CustomerID','Unnamed: 0','InvoiceNo','year','month','day','hour','UnitPrice'],axis=1)
type_customer_df = round(type_customer_df)

# To check % of amount spent by customer
total_amount = type_customer_df['amount_spent'].sum()
type_customer_df['%Amount_spent'] = round(type_customer_df['amount_spent'] / total_amount *100)


# To check quantiles
type_customer_df['Number_customers'] = df['Quantiles'].value_counts()
number_customers = df['CustomerID'].count()
type_customer_df['%Number_customers'] = round(type_customer_df['Number_customers'] / number_customers *100)

# 5% of customers spent 50% of revenue
type_customer_df


,Quantity,amount_spent,%Amount_spent,Number_customers,%Number_customers
Quantiles,,,,,
Low_tear,1155119,1859862.0,21.0,3254,75.0
Preferred,1541090,2562145.0,29.0,868,20.0
VIP,2485487,4489400.0,50.0,217,5.0


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [22]:
# your code here
# Table with quantile information per customer
customers_quantiles = df.drop(['Unnamed: 0','InvoiceNo','year','month','day','hour','UnitPrice','Quantity','amount_spent'],axis=1)
customers_quantiles

,CustomerID,Quantiles
0,12346,VIP
1,12347,Preferred
2,12348,Preferred
3,12349,Preferred
4,12350,Low_tear
...,...,...
4334,18280,Low_tear
4335,18281,Low_tear
4336,18282,Low_tear
4337,18283,Preferred


In [44]:
# Merge orders with customers_quantiles
merge_df = orders.merge(customers_quantiles,left_on = 'CustomerID',right_on = 'CustomerID')

# Number of VIP customer per country
country_count_VIP_df = merge_df[merge_df['Quantiles']=='VIP'].groupby(['Country']).count()
country_count_VIP_df.iloc[:,0].sort_values(ascending=False)

Country
United Kingdom     84185
EIRE                7077
France              3290
Germany             3127
Netherlands         2080
Australia            898
Portugal             681
Switzerland          594
Spain                511
Norway               420
Channel Islands      364
Finland              294
Cyprus               248
Singapore            222
Japan                205
Sweden               198
Belgium               54
Denmark               36
Name: Unnamed: 0, dtype: int64

In [46]:
# Final answer
country_count_VIP_df.iloc[:,0].sort_values(ascending=False).head(1)

Country
United Kingdom    84185
Name: Unnamed: 0, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [57]:
# your code here
country_count_df = merge_df[merge_df['Quantiles'].isin(['VIP','Preferred'])].groupby(['Country']).count()
country_count_df.iloc[:,0].sort_values(ascending=False).head(1)

Country
United Kingdom    221635
Name: Unnamed: 0, dtype: int64